In [1]:
# Importing the NumPy library for numerical computations.
# NumPy is used for handling arrays and performing mathematical operations efficiently.
import numpy as np

# Importing Matplotlib's pyplot module for creating static visualizations.
# Matplotlib is commonly used for plotting graphs, such as stock price movements or option pricing models.
import matplotlib.pyplot as plt

# Importing the Plotly Graph Objects module for creating interactive visualizations.
# Plotly is often used for more advanced, dynamic, and interactive plots that can be shared or embedded in reports.
import plotly.graph_objects as go

In [2]:
class AmericanOption:
    def __init__(self, S0, K, T, r, sigma, N, option_type="call"):
        """
        Initialize the parameters for the American Option pricing model.

        Parameters:
        - S0: Initial stock price (float)
        - K: Strike price of the option (float)
        - T: Time to maturity in years (float)
        - r: Risk-free interest rate (float)
        - sigma: Volatility of the stock (float)
        - N: Number of time steps in the binomial model (int)
        - option_type: Type of the option ("call" or "put")
        """


        # Assign input parameters to class attributes
        self.S0 = float(S0)
        self.K = float(K)
        self.T = float(T)
        self.r = float(r)
        self.sigma = float(sigma)
        self.N = int(N)
        self.option_type = option_type.lower()


        # Calculate the size of each time step
        self.dt = self.T / self.N


        # Calculate up (u) and down (d) factors
        self.u = np.exp(self.sigma * np.sqrt(self.dt))
        self.d = 1 / self.u


        # Calculate the risk-neutral probability
        self.p = (np.exp(self.r * self.dt) - self.d) / (self.u - self.d)


        # Validate that the probability lies between 0 and 1
        if not (0 < self.p < 1):
            raise ValueError("Invalid parameters: Ensure 0 < p < 1 for risk-neutral probability.")



    def build_tree(self):
        """
        Build the binomial tree for the stock prices.

        Returns:
        - tree: A NumPy array representing the stock price tree
        """


        # Create an empty array to hold stock prices
        tree = np.zeros((self.N + 1, self.N + 1))


        # Populate the tree
        for i in range(self.N + 1):
            # Calculate stock prices at each node
            tree[:i + 1, i] = self.S0 * (self.u ** np.arange(i + 1)) * (self.d ** (i - np.arange(i + 1)))
        return tree




    def evaluate(self):
        """
        Evaluate the American Option using the binomial model.

        Returns:
        - option_price: The fair price of the option (float)
        - stock_tree: The binomial tree for stock prices (NumPy array)
        - option_values: The binomial tree for option values (NumPy array)
        """


        # Build the binomial tree for stock prices
        stock_tree = self.build_tree()


        # Create an array to store option values
        option_values = np.zeros_like(stock_tree)


        # Calculate the payoff at maturity for each node
        for j in range(self.N + 1):
            stock_price = stock_tree[j, self.N]
            if self.option_type == "call":
                # Payoff for a call option
                option_values[j, self.N] = max(stock_price - self.K, 0)
            elif self.option_type == "put":
                # Payoff for a put option
                option_values[j, self.N] = max(self.K - stock_price, 0)


        # Backward induction: Calculate option values at earlier nodes
        for i in range(self.N - 1, -1, -1): # Start from second-to-last column
            for j in range(i + 1): # Traverse each node in the column
                stock_price = stock_tree[j, i]

                # Calculate the continuation value (expected discounted payoff)
                continuation_value = np.exp(-self.r * self.dt) * (
                    self.p * option_values[j + 1, i + 1] + (1 - self.p) * option_values[j, i + 1]
                )

                # Calculate the intrinsic value of the option
                intrinsic_value = (
                    max(stock_price - self.K, 0) if self.option_type == "call" else max(self.K - stock_price, 0)
                )

                # American Option: Take the maximum of continuation and intrinsic value
                option_values[j, i] = max(continuation_value, intrinsic_value)  # American Option property



        # Return the option price, stock tree, and option value tree
        return option_values[0, 0], stock_tree, option_values




    def plot_tree(self, tree, title="Tree Visualization", tree_type="stock"):
        """
        Plot the binomial tree for stock prices or option values.

        Parameters:
        - tree: NumPy array representing the tree to be plotted
        - title: Title of the plot (string)
        - tree_type: Type of tree being plotted ("stock" or "option")
        """


        # Label the y-axis based on the tree type
        label = "Stock Price" if tree_type == "stock" else "Option Value"


        # Create a Plotly figure
        fig = go.Figure()


        # Add nodes to the plot
        for i in range(tree.shape[1]): # Iterate over columns
            for j in range(i + 1): # Iterate over rows in each column

                # Add a scatter plot for each node
                fig.add_trace(go.Scatter(x=[i], y=[tree[j, i]],
                                        text=[f"{tree[j, i]:.2f}"],
                                        mode="markers+text", textposition="top center"))


        # Update the layout of the plot
        fig.update_layout(title=title, xaxis_title="Time Steps", yaxis_title=label)


        # Display the plot
        fig.show()


In [3]:
if __name__ == "__main__":
    """
    Main execution block for testing the AmericanOption class.
    This script initializes an option, evaluates its price, and visualizes its trees.
    """


    # Define input parameters for the American Option
    S0 = 100       # Initial stock price
    K = 100        # Strike price
    T = 1          # Time to maturity (in years)
    r = 0.05       # Risk-free interest rate (annualized)
    sigma = 0.2    # Volatility of the stock
    N = 5          # Number of time steps in the binomial model
    option_type = "call"  # Type of the option ("call" or "put")



    # Create an instance of the AmericanOption class with the given parameters
    option = AmericanOption(S0, K, T, r, sigma, N, option_type)



    # Evaluate the American Option to compute its price
    option_price, stock_tree, option_tree = option.evaluate()



    # Print the computed option price
    print(f"The price of the American Option is: {option_price:.2f}")



    # Visualize the option value tree
    option.plot_tree(option_tree, title="Option Value Tree", tree_type="option")



    # Visualize the stock price tree
    option.plot_tree(stock_tree, title="Stock Price Tree", tree_type="stock")


The price of the American Option is: 10.81


In [5]:
class AmericanBarrierOption:
    def __init__(self, S0, K, T, r, sigma, N, barrier, option_type="call", barrier_type="up-and-out"):
        """
        Initialize the parameters for the American Option pricing model.

        Parameters:
        - S0: Initial stock price (float)
        - K: Strike price of the option (float)
        - T: Time to maturity in years (float)
        - r: Risk-free interest rate (float)
        - sigma: Volatility of the stock (float)
        - N: Number of time steps in the binomial model (int)
        - barrier: Barrier level
        - option_type: Type of the option ("call" or "put")
        - barrier type: "up-and-out" or "down-and-out"
        """


        # Initialize option parameters
        self.S0 = float(S0)
        self.K = float(K)
        self.T = float(T)
        self.r = float(r)
        self.sigma = float(sigma)
        self.N = int(N)
        self.barrier = float(barrier)
        self.option_type = option_type.lower()
        self.barrier_type = barrier_type.lower()



        # Calculate the size of each time step
        self.dt = self.T / self.N



        # Calculate up (u) and down (d) factors
        self.u = np.exp(self.sigma * np.sqrt(self.dt))
        self.d = 1 / self.u



        # Calculate the risk-neutral probability
        self.p = (np.exp(self.r * self.dt) - self.d) / (self.u - self.d)



        # Validate that the probability lies between 0 and 1
        if not (0 < self.p < 1):
            raise ValueError("Invalid parameters: Ensure 0 < p < 1 for risk-neutral probability.")



    def build_tree(self):
        """
        Build the binomial tree for the stock prices, incorporating the barrier.

        Returns:
        - tree: A NumPy array representing the stock price tree
        """


        # Create an empty array to hold stock prices
        tree = np.zeros((self.N + 1, self.N + 1))


        # Populate the tree
        for i in range(self.N + 1):
            for j in range(i + 1):
                stock_price = self.S0 * (self.u ** j) * (self.d ** (i - j))


                # Invalidate nodes that cross the barrier
                if (self.barrier_type == "up-and-out" and stock_price >= self.barrier) or \
                   (self.barrier_type == "down-and-out" and stock_price <= self.barrier):
                    tree[j, i] = np.nan  # Mark as invalid
                else:
                    tree[j, i] = stock_price  #Set valid stock price



        return tree




    def evaluate(self):
      """
      Evaluate the American Barrier Option using the binomial model.

      Returns:
      - option_price: The fair price of the option (float)
      - stock_tree: The binomial tree for stock prices (NumPy array)
      - option_values: The binomial tree for option values (NumPy array)
      """


      # Build the binomial tree for stock prices
      stock_tree = self.build_tree()


      # Create an array to store option values
      option_values = np.zeros_like(stock_tree)



      # Calculate the payoff at maturity for each node
      for j in range(self.N + 1):
          stock_price = stock_tree[j, self.N]
          if not np.isnan(stock_price):  # Ensure the node is valid
              intrinsic_value = (
                  max(stock_price - self.K, 0) if self.option_type == "call"
                  else max(self.K - stock_price, 0)
              )
              option_values[j, self.N] = intrinsic_value



      # Backward induction: Calculate option values at earlier nodes
      for i in range(self.N - 1, -1, -1): # Start from second-to-last column
          for j in range(i + 1): # Traverse each node in the column
              stock_price = stock_tree[j, i]



              if not np.isnan(stock_price):  # Skip nodes that violate the barrier
                  # Calculate the continuation value (expected discounted payoff)
                  continuation_value = np.exp(-self.r * self.dt) * (
                      self.p * option_values[j + 1, i + 1] +
                      (1 - self.p) * option_values[j, i + 1]
                  )


                  # Calculate the intrinsic value of the option
                  intrinsic_value = (
                      max(stock_price - self.K, 0) if self.option_type == "call"
                      else max(self.K - stock_price, 0)
                  )



                  # American Option: Take the maximum of continuation and intrinsic value
                  option_values[j, i] = max(continuation_value, intrinsic_value)


      # Return the option price, stock tree, and option value tree
      return option_values[0, 0], stock_tree, option_values



    def plot_tree_with_barrier(self, tree, title="Stock Price Tree"):
      """
      Plot the binomial stock price tree with a barrier.

      Parameters:
      - tree: NumPy array representing the stock price tree, where each element corresponds
              to a stock price at a specific node in the tree.
      - title: Title of the plot (string). Default is "Stock Price Tree".
      """



      # Initialize an empty plotly figure object
      fig = go.Figure()


      # Loop through the columns of the tree (time steps)
      for i in range(tree.shape[1]):


          # Loop through the rows of each column (stock prices at that time step)
          for j in range(i + 1):


              # Get the stock price at node (j, i)
              stock_price = tree[j, i]


              # Process only valid nodes (non-NaN values)
              if not np.isnan(stock_price):
                  # If the stock price exceeds the barrier, display it as 0
                  displayed_price = stock_price if stock_price < self.barrier else 0
                  # Add a scatter point for the stock price
                  fig.add_trace(go.Scatter(
                      x=[i], y=[displayed_price],
                      text=[f"{displayed_price:.2f}"],
                      mode="markers+text",
                      textposition="top center",
                      marker=dict(color="blue" if stock_price < self.barrier else "red"),
                  ))



      # Add a horizontal dashed line to represent the barrier level
      fig.add_shape(
          type="line",
          x0=0, x1=self.N,
          y0=self.barrier, y1=self.barrier,
          line=dict(color="Red", width=2, dash="dash"),
          name="Barrier"
      )



      # Customize the plot layout
      fig.update_layout(
          title=title,
          xaxis_title="Time Steps",
          yaxis_title="Stock Price",
          showlegend=False
      )


      # Display the plot in the browser
      fig.show()


In [6]:
if __name__ == "__main__":
    """
    Main execution block for testing the AmericanBarrierOption class.
    This script initializes an option, evaluates its price, and visualizes its trees.
    """



    # Define the parameters for the option
    S0 = 100           # Initial stock price
    K = 100            # Strike price
    T = 1              # Time to maturity (in years)
    r = 0.05           # Risk-free interest rate (annualized)
    sigma = 0.2        # Volatility of the underlying stock (annualized)
    N = 5              # Number of time steps in the binomial tree
    barrier = 120      # Barrier level
    option_type = "call"  # Type of option ("call" or "put")
    barrier_type = "up-and-out"  # Type of barrier ("up-and-out", "down-and-out", etc.)



    # Create an AmericanBarrierOption object with the specified parameters
    option = AmericanBarrierOption(S0, K, T, r, sigma, N, barrier, option_type, barrier_type)



    # Evaluate the option to calculate its price, stock tree, and option value tree
    option_price, stock_tree, option_tree = option.evaluate()



    # Print the calculated price of the American Barrier Option
    print(f"The price of the American Barrier Option is: {option_price:.2f}")



    # Plot the stock price tree with the barrier
    option.plot_tree_with_barrier(stock_tree, title="Stock Price Tree with Barrier")



    # Plot the option value tree with the barrier
    option.plot_tree_with_barrier(option_tree, title="Option Value Tree with Barrier")

The price of the American Barrier Option is: 9.61
